In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


###Import Libraries

In [ ]:
import pandas as pd
import numpy as np

import spacy
nlp = spacy.load('en_core_web_sm')

from spacy.lang.en.stop_words import STOP_WORDS

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import naive_bayes, svm
from sklearn.metrics import accuracy_score, recall_score, precision_score
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings("ignore")

In [ ]:
df = pd.read_csv('/content/gdrive/MyDrive/NLP Sentiment Project/data/Reviews.csv')

In [ ]:
df.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [ ]:
df = df.filter(['Score','Text'], axis=1)

In [ ]:
df.Score.value_counts()

5    363122
4     80655
1     52268
3     42640
2     29769
Name: Score, dtype: int64

In [ ]:
df = df.head(5000)

In [ ]:
df.Score.value_counts()

5    3134
4     712
1     469
3     395
2     290
Name: Score, dtype: int64

In [ ]:
df.to_csv('trial.csv')

In [ ]:
pd.set_option('display.max_colwidth', 200)

In [ ]:
corpus = pd.read_csv("trial.csv", delimiter=",", encoding='utf-8', header= 0, index_col= 0)

In [ ]:
corpus.shape

(5000, 2)

In [ ]:
corpus.head()

,Score,Text
0,5,I have bought several of the Vitality canned dog food products and have found them all to be of good quality. The product looks more like a stew than a processed meat and it smells better. My Labr...
1,1,"Product arrived labeled as Jumbo Salted Peanuts...the peanuts were actually small sized unsalted. Not sure if this was an error or if the vendor intended to represent the product as ""Jumbo""."
2,4,"This is a confection that has been around a few centuries. It is a light, pillowy citrus gelatin with nuts - in this case Filberts. And it is cut into tiny squares and then liberally coated with ..."
3,2,If you are looking for the secret ingredient in Robitussin I believe I have found it. I got this in addition to the Root Beer Extract I ordered (which was good) and made some cherry soda. The fl...
4,5,"Great taffy at a great price. There was a wide assortment of yummy taffy. Delivery was very quick. If your a taffy lover, this is a deal."


In [ ]:
corpus.isnull().sum()

Score    0
Text     0
dtype: int64

In [ ]:
corpus.columns = ['label', 'text'] 

###Hugging Face

In [ ]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 14.9 MB/s 
     |████████████████████████████████| 596 kB 59.1 MB/s 
     |████████████████████████████████| 6.6 MB 62.3 MB/s 
     |████████████████████████████████| 101 kB 12.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import logging
import sys
from transformers import pipeline

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("LiYuan/amazon-review-sentiment-analysis")

model = AutoModelForSequenceClassification.from_pretrained("LiYuan/amazon-review-sentiment-analysis")

Downloading:   0%|          | 0.00/556 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/851k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.44M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/638M [00:00<?, ?B/s]

In [ ]:
from transformers import pipeline
import torch
import torch.nn.functional as F

####Here I used pytorch instead of pipeline

In [ ]:
for index, entry in enumerate(corpus['text']):
  batch = tokenizer(entry, padding = True, truncation = True, max_length = 512, return_tensors = "pt")
  with torch.no_grad():
    outputs = model(**batch)
    predictions = F.softmax(outputs.logits, dim =1 )
    labels = torch.argmax(predictions, dim = 1)
    labels = [model.config.id2label[label_id] for label_id in labels.tolist()]
    corpus.loc[index,'labels'] = labels

#CAUTION: This needs to run for 1 hour 17 minutes !!!

In [ ]:
corpus = corpus.reindex(columns = ['label','labels','text'])
corpus.columns = ['label', 'label_hf','text'] 
corpus.head()

,label,label_hf,text
0,5,5 stars,I have bought several of the Vitality canned dog food products and have found them all to be of good quality. The product looks more like a stew than a processed meat and it smells better. My Labr...
1,1,1 star,"Product arrived labeled as Jumbo Salted Peanuts...the peanuts were actually small sized unsalted. Not sure if this was an error or if the vendor intended to represent the product as ""Jumbo""."
2,4,5 stars,"This is a confection that has been around a few centuries. It is a light, pillowy citrus gelatin with nuts - in this case Filberts. And it is cut into tiny squares and then liberally coated with ..."
3,2,5 stars,If you are looking for the secret ingredient in Robitussin I believe I have found it. I got this in addition to the Root Beer Extract I ordered (which was good) and made some cherry soda. The fl...
4,5,5 stars,"Great taffy at a great price. There was a wide assortment of yummy taffy. Delivery was very quick. If your a taffy lover, this is a deal."


In [ ]:
import re
import regex

In [ ]:
number = lambda x: re.search(r'\d+', x).group()
corpus['label_hf'] = corpus['label_hf'].apply(number)

In [ ]:
corpus.head(20)

,label,label_hf,text
0,5,5,I have bought several of the Vitality canned dog food products and have found them all to be of good quality. The product looks more like a stew than a processed meat and it smells better. My Labr...
1,1,1,"Product arrived labeled as Jumbo Salted Peanuts...the peanuts were actually small sized unsalted. Not sure if this was an error or if the vendor intended to represent the product as ""Jumbo""."
2,4,5,"This is a confection that has been around a few centuries. It is a light, pillowy citrus gelatin with nuts - in this case Filberts. And it is cut into tiny squares and then liberally coated with ..."
3,2,5,If you are looking for the secret ingredient in Robitussin I believe I have found it. I got this in addition to the Root Beer Extract I ordered (which was good) and made some cherry soda. The fl...
4,5,5,"Great taffy at a great price. There was a wide assortment of yummy taffy. Delivery was very quick. If your a taffy lover, this is a deal."
5,4,5,"I got a wild hair for taffy and ordered this five pound bag. The taffy was all very enjoyable with many flavors: watermelon, root beer, melon, peppermint, grape, etc. My only complaint is there wa..."
6,5,5,"This saltwater taffy had great flavors and was very soft and chewy. Each candy was individually wrapped well. None of the candies were stuck together, which did happen in the expensive version, ..."
7,5,5,This taffy is so good. It is very soft and chewy. The flavors are amazing. I would definitely recommend you buying it. Very satisfying!!
8,5,5,Right now I'm mostly just sprouting this so my cats can eat the grass. They love it. I rotate it around with Wheatgrass and Rye too
9,5,5,This is a very healthy dog food. Good for their digestion. Also good for small puppies. My dog eats her required amount at every feeding.


In [ ]:
corpus['label_hf'] = corpus['label_hf'].astype('int64')

In [ ]:
corpus.dtypes

label        int64
label_hf     int64
text        object
dtype: object

##Evaluation

In [ ]:
from sklearn.metrics import accuracy_score, mean_absolute_error, f1_score

In [ ]:
mean_absolute_error(corpus['label'], corpus['label_hf'])

0.3682

In [ ]:
accuracy_score(corpus['label'], corpus['label_hf'])

0.7444

In [ ]:
f1_score(corpus['label'], corpus['label_hf'], average = None)

array([0.69375   , 0.38857143, 0.41991925, 0.38023451, 0.88294314])

In [ ]:
f1_score(corpus['label'], corpus['label_hf'], average = 'micro')

0.7444